# Лабораторна робота №8
### Нечіткий метод групового урахування аргументів (МГУА)

Прогнозування індексу споживчих цін (ІСЦ) з використанням нечіткого підходу.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# Завантаження попередньо конвертованого CSV-файлу
file_path = 'data5.csv'
df = pd.read_csv(file_path)
df = df.dropna()
df.head()

In [ ]:
# Вибираємо цільову змінну для прогнозу
y = df['ІСЦ (+1)'].astype(float).values

In [ ]:
def create_lagged_matrix(y, lag):
    X, Y = [], []
    for i in range(lag, len(y) - 5):
        X.append(y[i - lag:i])
        Y.append(y[i:i + 5])
    return np.array(X), np.array(Y)

In [ ]:
def fuzzy_gmdh_model(X_train, Y_train, X_test):
    # Просте наближення нечіткого правила через LR на кожен крок
    models = [LinearRegression().fit(X_train, Y_train[:, i]) for i in range(5)]
    predictions = np.column_stack([m.predict(X_test) for m in models])
    return predictions

In [ ]:
def run_fuzzy_mgua(y, lag=4, num_models=5):
    mse = []
    for window_start in range(10):
        y_window = y[window_start:window_start + 100]
        X, Y = create_lagged_matrix(y_window, lag)

        split = len(X) // 2
        X_train, Y_train = X[:split], Y[:split]
        X_test, Y_test = X[split:], Y[split:]

        preds = fuzzy_gmdh_model(X_train, Y_train, X_test)
        mse.append(mean_squared_error(Y_test, preds))
    return mse

In [ ]:
def plot_mse(mse, num_models):
    plt.figure(figsize=(10, 5))
    plt.plot(mse, label='Без адаптації')
    plt.title(f'MSE на 5 точок уперед, нечіткий МГУА, моделей: {num_models}')
    plt.xlabel('Номер вікна')
    plt.ylabel('СКВ (MSE)')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
results = {}
for models in [5, 6, 7]:
    mse = run_fuzzy_mgua(y, lag=4, num_models=models)
    results[models] = mse
    plot_mse(mse, models)

In [ ]:
for models, mse in results.items():
    df_out = pd.DataFrame({
        'Window': list(range(1, len(mse)+1)),
        'MSE': mse
    })
    df_out.to_excel(f'FGMDH_результати_{models}_моделей.xlsx', index=False)